<a href="https://colab.research.google.com/github/buddhahacker/GreeterDistilled/blob/master/PDFSearchV17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=9b6454d00f04ac2c2c23be639fe5a3c5f647cd37e7279ab30c7a4e7e13a6976f
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [3]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

from os import chdir
import pandas as pd
from datetime import datetime, timedelta
from time import time
import PyPDF2
import re
import os


import datetime as dt
from multiprocessing import Process, current_process
import sys



####################################################################################
############################### DIRECTORY LOCATION FOR FILES #######################
####################################################################################

# Steps:
# 1 - Program base research directory below.
# 2 - Enter the directory to be search in the DirectoryToSearch.rtf file under the directoryvar directory defined below.
# 3 - Ensure that directory has both "input" and "output" directories.
# 4 - Ensure the search directory contains the "SearchWordsToExclude.rtf" and "TermsToSearch.rtf" files.
# 5 - Insert the epub, txt and PDF files to be searched.
# 6 - Decide if you want to run automated, manual or semi-automated.
# 7 - Run the program and your results will be contianed in the output directory.  Including converted PDF files, summary of terms/directory searched and results.
# 8 - The only known acceptable abend is due to a PDF file that is simply a concatination of images with no text.  You will need to do an OCR before using this program.


directoryvar = '/Users/freethebuddha/cloudstation/research'   #TODO BIG LOOP PROBLEM  must have a directory set in order to know where the DirectoryToSearch file exists


                                                            

chdir(directoryvar)
searchdirectory = open("DirectoryToSearch.rtf").read()         # TODO need to check if directories input and output exist


directory=searchdirectory.split(":")[1]
directoryvar=directory.split("}")[0]

####################################################################################
###############################  PROCEDURES ########################################
####################################################################################

##############################  epub Conversion  ######################################
#######   
####### Input: 
#######     PDFfile: The PDF file to be converted
#######
####### Output:
#######     A clear text file named 'data.txt' which will be used by the word search processes
#######
####### Internal variables etc.:
#######     Several for the PDF conversion process
#######         names are based on score counts (Low, Med, Hi)
####### Global variables etc.:
#######
#######
####### Purpose: Take the text where terms were found, these locations and  output
#######         to the appropriate files(see file name structure)
####################################################################################
import urllib.request, urllib.parse, urllib.error
import zipfile

import xml.parsers.expat
import html2text
from glob import glob



class ContainerParser():
  def __init__(self,xmlcontent=None):
    self.rootfile = ""  
    self.xml = xmlcontent

  def startElement(self, name, attributes):
    if name == "rootfile": 
      self.buffer = ""    
      self.rootfile = attributes["full-path"]

  def parseContainer(self):
    parser = xml.parsers.expat.ParserCreate()
    parser.StartElementHandler = self.startElement
    parser.Parse(self.xml, 1)
    return self.rootfile

class BookParser():
  def __init__(self,xmlcontent=None):
    self.xml = xmlcontent 
    self.title = "" 
    self.author = "" 
    self.inTitle = 0
    self.inAuthor = 0
    self.ncx = ""

  def startElement(self, name, attributes):
    if name == "dc:title":
      self.buffer = ""
      self.inTitle = 1 
    elif name == "dc:creator":
      self.buffer = ""
      self.inAuthor = 1 
    elif name == "item":
      if attributes["id"] == "ncx" or attributes["id"] == "toc" or attributes["id"] == "ncxtoc":
        self.ncx = attributes["href"]

  def characters(self, data):
    if self.inTitle:
      self.buffer += data
    elif self.inAuthor:
      self.buffer += data

  def endElement(self, name):
    if name == "dc:title":
      self.inTitle = 0  
      self.title = self.buffer  
      self.buffer = ""
    elif name == "dc:creator":
      self.inAuthor = 0  
      self.author = self.buffer  
      self.buffer = ""

  def parseBook(self):
    parser = xml.parsers.expat.ParserCreate()
    parser.StartElementHandler = self.startElement
    parser.EndElementHandler = self.endElement
    parser.CharacterDataHandler  = self.characters
    parser.Parse(self.xml, 1)
    return self.title,self.author, self.ncx

class NavPoint():
  def __init__(self,id=None,playorder=None,level=0,content=None,text=None):
    self.id = id 
    self.content = content
    self.playorder = playorder
    self.level = level
    self.text = text

class TocParser():
  def __init__(self,xmlcontent=None):
    self.xml = xmlcontent 
    self.currentNP = None
    self.stack = []
    self.inText = 0
    self.toc = []

  def startElement(self, name, attributes):
    if name == "navPoint":
      level = len(self.stack)
      self.currentNP = NavPoint(attributes["id"], attributes["playOrder"], level)
      self.stack.append(self.currentNP)
      self.toc.append(self.currentNP) 
    elif name == "content":
      self.currentNP.content = urllib.parse.unquote(attributes["src"])
    elif name == "text":
      self.buffer = ""
      self.inText = 1

  def characters(self, data):
    if self.inText:
      self.buffer += data

  def endElement(self, name):
    if name == "navPoint":
      self.currentNP = self.stack.pop()
    elif name == "text":
      if self.inText and self.currentNP:
        self.currentNP.text = self.buffer
      self.inText = 0  

  def parseToc(self):
    parser = xml.parsers.expat.ParserCreate()
    parser.StartElementHandler = self.startElement
    parser.EndElementHandler = self.endElement
    parser.CharacterDataHandler  = self.characters
    parser.Parse(self.xml, 1)
    return self.toc

class epub2txt():
  def __init__(self,epubfile=None):
    self.epub = epubfile  
    
    

  def convert(self):
    epubtext = ' '
    print("Processing %s ..." % self.epub)
    file = zipfile.ZipFile(self.epub,"r")
    rootfile = ContainerParser(file.read("META-INF/container.xml")).parseContainer()
    title, author, ncx = BookParser(file.read(rootfile)).parseBook()
    ops = "/".join(rootfile.split("/")[:-1])
    if ops != "":
      ops = ops+"/"
    toc = TocParser(file.read(ops + ncx)).parseToc()

    #fo = open("%s_%s.txt" % (title, author), "w")         commented out to prevent writing out file 
    for t in toc:
      html = file.read(ops + t.content.split("#")[0])
      text = html2text.html2text(html.decode("utf-8"))
      epubtext = epubtext + text
    
      #fo.write(text + "\n")
    #fo.close()
    file.close()
    return(epubtext)
    



##############################  String Chop  ######################################
#######   
####### Input: 
#######     PDFfile: The PDF file name to be chopped
#######
####### Output:
#######     A clear text version of the file name minus the characters to be chopped at the end
#######
####### Internal variables etc.:
#######     s - a variable containing the passed input string and returned chopped string
#######     suffix - the string containing the passed input string of characters to chop off   
#######
####### Purpose: Input a string along with the characters to be chopped at the end of the string.
#######         Chopped off the characters passed and return the chopped string
####################################################################################
def rchop(s, suffix):
    if suffix and s.endswith(suffix):
        return s[:-len(suffix)]
    return s



##############################  PDF Conversion  ######################################
#######   
####### Input: 
#######     PDFfile: The PDF file to be converted
#######
####### Output:
#######     A clear text file named 'data.txt' which will be used by the word search processes
#######
####### Internal variables etc.:
#######     Several for the PDF conversion process
#######         names are based on score counts (Low, Med, Hi)
#######
####### Purpose: Take the text where terms were found, these locations and  output
#######         to the appropriate files(see file name structure)
####################################################################################



def PDFConvert(PDFfile):
    
    from pdfminer3.layout import LAParams, LTTextBox
    from pdfminer3.pdfpage import PDFPage
    from pdfminer3.pdfinterp import PDFResourceManager
    from pdfminer3.pdfinterp import PDFPageInterpreter
    from pdfminer3.converter import PDFPageAggregator
    from pdfminer3.converter import TextConverter
    import io



    chdir(directoryvar+directoryinput)
    
    print('Converting PDF')

    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    
    with open(PDFfile, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                    caching=True,
                                    check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    


    chdir(directoryvar+directoryoutput) 
   
    somestring = PDFfile
    PDFfile = rchop(somestring, '.pdf')
    
    f = open(PDFfile+'.txt', 'w')   #TODO Fix this by removing .pdf extention and adding .txt
    print(text, file=f)
    
    f.close()


    return (text)  
    



def ProcessTextFile(pdfdata,chunksize):
###################### PROCESSING OF TEXT FILE ##############################
#######   
####### Input: 
#######     pdfdata: 
#######     
#######
####### Output:
#######     text: 
#######
####### Internal variables etc.:
#######     ***.txt - 
####### Global variables etc.:
#######     n      
#######
####### Purpose: 
#######         
####################################################################################
    

 
  

    chdir(directoryvar)

    TotalScore = 0
    i = 0           #Index
    s = 0           #Index
    z = 0           #Index
    hits ={}
    charpos = 0     # Initializing the character capturing loop to the first position
    PDFtext ={}
    SavedHits =[]
    Score=0         # Score as accumulated during the file search


    Fname = n       # from manual input of file name



    print('chunksize: ',chunksize)
    



    maxchar=len(pdfdata)
    finalpass=False
    print('maxchar', maxchar)
    
    





    while charpos+chunksize < maxchar+1:
        Score=0
        Hit=1
        DHit=2
        
        

        chunk= pdfdata[charpos:charpos+chunksize]  #changed from achunk to pdfdata
        
        

        #######  Search text inc score by 1 or 2
        
            #Search using 1 point SearchTerms
        
    
        
        for x in range(0,len(SearchTerms)):
            if SearchTerms[x] in chunk:   # TODO ADD a copy of the SearchTerms list with counter added. Increment as found
                Score+=Hit
                
    

            #Search using 2 point SearchTerms   INCLUDES SCORE FROM SEARCHTERMS********************
        for x in range(0,len(DSearchTerms)):
            if DSearchTerms[x] in chunk: 
                Score+=DHit
                
                
        if Score > 0:  
            ###limiter = str('@')
            ###SavedHits.append(limiter+Score+limiter)      # TODO look into putting an identifier after score number but having issue with str
            SavedHits.append(Score)
            SavedHits.append(charpos)
            SavedHits.append(chunk)
        

        charpos=charpos+chunksize
        if not finalpass:
            if charpos+chunksize > maxchar: 
                chunksize=maxchar-charpos
                finalpass=True
        

        TotalScore=TotalScore+Score 
    
    HitsLow = {}
    HitsMed = {}
    HitsHi = {}
    HLScore = 0
    HMScore = 0
    HHScore = 0

    while z in range(0,len(SavedHits)):
        
        
        if SavedHits[z] <ScoreValLow or SavedHits[z] is ScoreValLow: 
            HitsLow[SavedHits[z+1]] = str(SavedHits[z])+"@"+SavedHits[z+2]
            HLScore = HLScore + SavedHits[z]
            
            
        
        if SavedHits[z] >ScoreValLow and SavedHits[z] < ScoreValHi: 
            HitsMed[SavedHits[z+1]] = str(SavedHits[z])+"@"+SavedHits[z+2]
            HMScore = HMScore + SavedHits[z]
            

        if SavedHits[z] > ScoreValHi or SavedHits[z] is ScoreValHi: 
            HitsHi[SavedHits[z+1]] = str(SavedHits[z])+"@"+SavedHits[z+2]
            HHScore = HHScore + SavedHits[z]
            
            
        z+=3 ### Increment 3 (Score, CharacterPosition, Chunk)
    







    return(HitsLow,HitsMed,HitsHi,HLScore,HMScore,HHScore,TotalScore,maxchar)












##############################  writingfiles  ######################################
#######   
####### Input: 
#######     Hits: Dictionary containing text and location where terms were found
#######     filetype: String containing if Low Med or Hi should be used in file name
#######
####### Output:
#######     Clear text files using naming convention below in Internal variables etc.
#######
####### Internal variables etc.:
#######     Hits***.txt - output files with location and text where found. File
#######         names are based on score counts (Low, Med, Hi)
#######
####### Purpose: Take the text where terms were found, these locations and  output
#######         to the appropriate files(see file name structure)
####################################################################################
def writingfiles(Fname,Hits,filetype,typescore,maxchar):
    
    #directoryvar = '/Users/freethebuddha/cloudstation/Development/test'
    #directoryoutput = '/output'

    chdir(directoryvar+directoryoutput)
    
    
    
    
    print('I am writing files')
    # current date and time
    now = datetime.now()
    timestamp = now.strftime("Date:%m-%d  Time:%H-%M")
    # dd/mm/YY H:M:S format
    print("Timestamp:", timestamp)

    
    with open("%s%s%s%s%s%s%d%s%d.txt" %(Fname,'--', timestamp, '--', filetype, " Score:", typescore, '--',TotalScore), "w") as f:    
        for key in Hits:
            
            
            V=str(Hits[key])
           
            
            
            
            f.write("\n\n\n")
            f.write("%s%d%s%d%s%s%d\n"%('Location:',key,' of ',maxchar, '     ', 'Total Score:', TotalScore))
            
            for x in range(0,len(V)):
                if V[x] is '@':
                    scorelocation = x
            
            i=0
            
            f.write("The score of this chunk is ")
            while i < scorelocation:
                #i+=1    when looking for delimiter prior to score amount
                f.write(V[i])
                i+=1
                
            #chdir("/Users/freethebuddha/cloudstation/Development/Test/Output")   #TODO remove since the directory is not changed from initialization of this proc 
            f.write("\n\n\n")
            f.write("%s\n\n"%'**************************************************')
            
            
            f.write("%s\n"%V)
    f.close()


#######  Getting words from file to use in "getSearchWords"  function ###########
def getWords(text):
    return re.compile(r'\w+').findall(text)



##############################  getSearchWords  #####################################
#######  Grab Search Terms ######## Search text in file must begin w/word "SEARCH"
####### Input: 
#######     STerms: String containing search terms or the value 'n'
#######    
####### Output:
#######     Two lists: SWords and DSWords
#######
####### Internal variables etc.:
#######     SWords - List of search terms containing single point terms
#######     DSWords - list of search terms containing double point terms
#######
####### Purpose: Passed a string.  If the first character is 'n' then the search
#######     terms will be contained in the file "TermsToSearch.rtf".  Otherwise, the 
#######     terms are are in the list passed.  Parse the list to create a individual
#######     lists of single point search words and double point search words.
#######     -- Terms less than 2 characters are removed
#######     -- Connector words are removed
#######     -- Duplicate Terms are removed
#######     -- Terms greater than 4 characters or are capitalized are put in the 
#######         double point list
#######     -- Terms that are 3 and 4 characters and are not capitalized are placed
#######         single point list
####################################################################################
def getSearchWords(STerms):

    

    chdir(directoryvar)
   
    
    SWords = []
    DSWords = []
    ConnectorWords = []
    
    
    if STerms[0] is 'n':
    
        searchfile = open("TermsToSearch.rtf").read()
        searchwords = getWords(searchfile)
        
    # Identify the loaction of the word Search which deliniates where terms start
        location=searchwords.index('Search')

    # Grab search terms from text file after the word Search
        for i in range(location+1, len(searchwords)): 
            SWords.insert(i,searchwords[i])
    else:
        SWords=STerms

    

    

    # Places terms with more than 4 characters into the DoubleSWords(DSWords)
    #TODO should I move this to after the removal of all of the duplicates???
    for element in SWords:
        if len(element) > 4: DSWords.append(element)
        #TODO should I then remove it from the SWords list????


    # Removes terms less than 3 characters                                   #changed from 2 to 3 5-19-20
    for element in SWords:
        if len(element) <3: SWords.remove(element)

    # Removes Duplicates
    mylist = SWords
    SWords = list(dict.fromkeys(mylist)) 


    # Removes connector words - ConnectorWords are contained in the file below:

    ConnectorWordFile = open('SearchWordsToExclude.rtf').read()
    CWords = getWords(ConnectorWordFile)
    location=CWords.index('Exclude')
        # Grab search terms from text file after the word Search
    for i in range(location+1, len(CWords)): 
        ConnectorWords.insert(i,CWords[i])
   

    for element in SWords:
        if element in ConnectorWords: SWords.remove(element)

    # Removes terms less than 4 characters unless the are capitalize
    #TODO does this have duplicate effort to the removal above???
    for element in SWords:  
        
        if len(element) >1 and len(element) <4:
            
            if not re.findall('([A-Z]+)', element): 
                SWords.remove(element)
            
    
    # Captures capitalized terms with length of 3 or 4 chars and copies them to Double
    for element in SWords:
        
        if len(element) is 4 or len(element) is 3:
            
            
            if not re.findall('([A-Z]+)', element): 
                print()
                
            else: 
                DSWords.append(element)
                
                

    # Compares Double and if term in Search remove it from Search
    for element in DSWords:
       
        if element in SWords: SWords.remove(element)

    


    chdir(directoryvar+directoryoutput)
    
    Allsearchterms = []
    now = datetime.now()
    timestamp = now.strftime("%m-%d - %H-%M")
    with open("%s%s.txt" %('Search and Excluded Terms --', timestamp), "w") as f:   

        

        f.write("%s%s%s\n\n%s%s\n\n%s%s\n"%('Search Terms: ', SWords,DSWords, 'Excluded Connector Terms:', ConnectorWords, 'Directory: ', directoryvar))

         
    f.close()



    return (SWords,DSWords)





####################################################################################
##############################   MAINLINE  #########################################
####################################################################################
####### 
#######   STUB for requesting pdf file name to be search, file of what to be searched
#######   and the size of the chunking  
#######   Duplicate hits are not identified and therefore not counted
#######   Singles, Doubles score correctly.   
####### 
####################################################################################







####################################################################################
################################# INITIALIZATION ###################################
####################################################################################


TotalScore = 0  # Not currently being used
PDFtext ={}
SavedHits =[]
x = []          # Search phrase that is manually entered
pfile ='pdf'
efile ='epub'
tfile ='txt'
rfile ='rtf'

HitsL = {}
HitsM = {}
HitsH = {}

HLScore = 0
HMScore = 0
HHScore = 0

maxchar = 0

n = 'n'

ScoreValLow=2   # Low valued hits
ScoreValMed=4   # Not used right now
ScoreValHi=6 

chunksizedefault = int(800)


searchdirectory = open("DirectoryToSearch.rtf").read()


directory=searchdirectory.split(":")[1]
directoryvar=directory.split("}")[0]







#TODO fix this with DirectoryToSearch file input
directoryinput = '/input'
directoryoutput = '/output'




######################## RETRIEVE SEARCH TERMS ####################################

print('   Crawford Technologies Presents:  ')
print()
print('                                                PPPPPP     DDDD      FFFFFFF         ')
print('                                               P     P    D  DD     F  ')
print('                                              P     P    D    D    F')
print('                                             PPPPPP     D    D    FFFFF           ')
print('                                            P          D    D    F')
print('                                           P          D  DD     F')
print('                                          P          DDDD      F            ')
print('                      ')
print('                                 SSSSSS   EEEEEE       A       RRRRRR    CCCCCC   H    H')
print('                                S        E           A A      R    R    C        H    H')
print('                               SSSSSS   EEEEE      A   A     RRRRRR    C        HHHHHH')
print('                                   S   E         A AAA A    R  R      C        H    H')
print('                             SSSSSS   EEEEEE   A       A   R    R    CCCCCC   H    H')
print('                                           ')
print()
print('   with TECHNOLOGIES from Buddhahacker!')
print()

#from datetime import datetime  

# current date and time
now = datetime.now()
s2 = now.strftime("%d/%m/%Y, %H:%M:%S")
# dd/mm/YY H:M:S format
print("s2:", s2)



auto=input("Enter 'A' if you like to go AUTO on this?")

if auto != 'A':

    helpdefinitions = input("Enter 'Y' if you would like to see the default options?")

    if helpdefinitions is 'Y':
        print('# of chars searched in a chunk: ',chunksizedefault)
        print('The Low, Medium and High score values are: ', ScoreValLow, ScoreValMed, ScoreValHi)

    n=input("Enter file name to be searched WITH OUT extension or 'n' to to source names from directory (batch mode: ")

    if n != 'n':
        e=input("Enter 'e' 't' 'p' if source search material is txt or PDF format: ")      # TODO Remove and search for file type

    x=input("Enter phrase or terms to search in the PDF or enter 'n' to use the file: ")


    chunksizeINPUT=int(input("Enter the amount of text you would like scanned at a time or '0' for default(chunksizedefault): "))
                                                                                        # TODO Need to add logic around pdf file name

else:
    x = 'n'
    chunksizeINPUT=chunksizedefault

    
if x[0] is 'n':
    STerms = 'n'
else:
    STerms = x.split()

(SearchTerms,DSearchTerms) = getSearchWords(STerms)



if chunksizeINPUT is 0:
    chunksize=chunksizedefault 
else:
    chunksize=int(chunksizeINPUT)               #### CHUNK SIZE ****

if n != 'n':
    #fileN = n
    
    if e is 'p':
    
        PDFfilenametoconvert = n+'.pdf'                                         # TODO Need to redo the entire prompting for file type. Not neccessary?
        PDFtext=PDFConvert(PDFfilenametoconvert)                                            # TODO fix for passing data between procs rather than using files. See below:
        (HitsL,HitsM,HitsH,HLScore,HMScore,HHScore,TotalScore,maxchar) = ProcessTextFile(PDFtext,chunksize)
        ####################  Writing Output to Files   #####################
        
        writingfiles(PDFfilenametoconvert,HitsL,'L',HLScore,maxchar)
        writingfiles(PDFfilenametoconvert,HitsM,'M',HMScore,maxchar)
        writingfiles(PDFfilenametoconvert,HitsH,'H',HHScore,maxchar)

    if e is 't':

        filename = n+'.txt'
        f=open(filename, "r")                                
        if f.mode == 'r':
            PDF_Text =f.read()                                 

                             
        print('txt converted')
        print('chunk size: ',chunksize)
        chdir(directoryvar)
        (HitsL,HitsM,HitsH,HLScore,HMScore,HHScore,TotalScore,maxchar) = ProcessTextFile(PDF_Text,chunksize) 
       
       
        (HitsL,HitsM,HitsH,HLScore,HMScore,HHScore,TotalScore,maxchar) = ProcessTextFile(PDF_Text,chunksize)
        ####################  Writing Output to Files   #####################
        writingfiles(filename,HitsL,'L',HLScore,maxchar)
        writingfiles(filename,HitsM,'M',HMScore,maxchar)
        writingfiles(filename,HitsH,'H',HHScore,maxchar)




                    # TODO make this as part of the else above

if n is 'n':


    chdir(directoryvar+directoryinput)
    
    print('enter mailine')
    g=0
    worker_count = 8
    worker_pool = []
    PDF_Text = {}
    

    arr = os.listdir('.')
    print('Processing ', len(arr), 'files.')
    print(arr)


    while g in range (0,len(arr)):
        fileentry = arr[g]
        print('filename: ', fileentry)
        HLScore = 0
        HMScore = 0
        HHScore = 0
        print('Processing file ', g+1, 'of ', len(arr))
        
        if '.pdf' in fileentry:                                                 # added '.' in order to get only file
            PDFfilenametoconvert = arr[g]
            PDF_Text = PDFConvert(PDFfilenametoconvert)                         # convert pdf to text
            print('pdf converted')
            
            chdir(directoryvar)
            (HitsL,HitsM,HitsH,HLScore,HMScore,HHScore,TotalScore,maxchar) = ProcessTextFile(PDF_Text,chunksize)             # search text file
        
        
            writingfiles(fileentry,HitsL,'L',HLScore,maxchar)                                       # write out results
            writingfiles(fileentry,HitsM,'M',HMScore,maxchar)
            writingfiles(fileentry,HitsH,'H',HHScore,maxchar)
        
        
        else:
            print('not pdf')
        
        if 'txt' in fileentry:                                                      
            chdir(directoryvar+directoryinput)
            filename = fileentry
            f=open(filename, "r")

                                                    
            if f.mode == 'r':
                book =f.read()                                 

            PDF_Text = book                         
            print('txt converted')
            print('chunk size: ',chunksize)
            chdir(directoryvar)
            (HitsL,HitsM,HitsH,HLScore,HMScore,HHScore,TotalScore,maxchar) = ProcessTextFile(PDF_Text,chunksize)              # search text file
        
        
            writingfiles(fileentry,HitsL,'L',HLScore,maxchar)                                       # write out results
            writingfiles(fileentry,HitsM,'M',HMScore,maxchar)
            writingfiles(fileentry,HitsH,'H',HHScore,maxchar)

        else:
            print('not text')
        

        if 'epub' in fileentry: 
            chdir(directoryvar+directoryinput)
            epubfilenametoconvert = arr[g] 
            epubtext = epub2txt(epubfilenametoconvert).convert() 
            print('epub converted') 
 
            chdir(directoryvar) 
            (HitsL,HitsM,HitsH,HLScore,HMScore,HHScore,TotalScore,maxchar) = ProcessTextFile(epubtext,chunksize)  

            writingfiles(fileentry,HitsL,'L',HLScore,maxchar)                                       # write out results 
            writingfiles(fileentry,HitsM,'M',HMScore,maxchar) 
            writingfiles(fileentry,HitsH,'H',HHScore,maxchar) 
        
        else:
            print('not epub')



        g+=1
    
    
    
    n = "stop"




FileNotFoundError: ignored